In [1]:
import requests
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Setup splinter

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\ccrom\.wdm\drivers\chromedriver\win32\99.0.4844.51]


## NASA MARS NEWS

In [ ]:
# URL to visit
url = 'https://redplanetscience.com/'
browser.visit(url)

# Scrape the latest News Title & Paragraph Text 
html = browser.html
soup = bs(html, 'html.parser')

news_title = soup.find('div', class_='content_title').text

news_p = soup.find('div', class_='article_teaser_body').text

print(f"Title: {news_title}")
print(f"Article: {news_p}")

# # # Quit browser once scraping is complete
# browser.quit()


## JPL Mars Space Image

In [ ]:
# Visit new site to find image jpg. for feature image
url = 'https://spaceimages-mars.com'
browser.visit(url)

# Scrape site for featured image jpg.
html = browser.html
soup = bs(html, 'html.parser')

results = soup.find_all('div', class_= 'floating_text_area')

for result in results:
    link = result.a['href']
    featured_image_url = f'https://spaceimages-mars.com/{link}'

    print(featured_image_url)
    
# browser.links.find_by_partial_text('FULL IMAGE').click()

## Mars Facts

In [ ]:
# Use pandas to scrape table from url
url = 'https://galaxyfacts-mars.com'

# read in tables, and print them
tables = pd.read_html(url)
tables

In [ ]:
# Index into list to see the tables we have: Mars & Earth Comparison
tables[0]

In [ ]:
# Index into second list item where we have Mars facts only (I prefer this one)
tables[1]

In [ ]:
# Save desired table to Variable df 
df = tables[1]
df

In [ ]:
# Saving desired table to html and not including the header or index
df.to_html('marsfact_table.html', header = False, index = False, classes= 'table')

## Mars Hemispheres

In [3]:
# Visit new site to find image jpg. for feature image
url = 'https://marshemispheres.com/'
browser.visit(url)

In [4]:
html = browser.html
soup = bs(html, 'html.parser')

results = soup.find_all('div', class_='description')

In [5]:
results

[<div class="description">
 <a class="itemLink product-item" href="cerberus.html">
 <h3>Cerberus Hemisphere Enhanced</h3>
 </a>
 <span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/>
 <p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p>
 </div>,
 <div class="description">
 <a class="itemLink product-item" href="schiaparelli.html">
 <h3>Schiaparelli Hemisphere Enhanced</h3>
 </a>
 <span class="subtitle" style="float:left">image/tiff 35 MB</span><span class="pubDate" style="float:right"></span><br/>
 <p>Mosaic of the Schiaparelli hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. The images were acquired in 1980 during early northern…</p>
 </div>,
 <div class="description">
 <a class="itemLink product-item" 

In [6]:
title_list = []

for result in results:
    title_txt = result.find('h3').text
    title = title_txt.split('Enhanced')[0]
    title_list.append(title)

In [7]:
title_list

['Cerberus Hemisphere ',
 'Schiaparelli Hemisphere ',
 'Syrtis Major Hemisphere ',
 'Valles Marineris Hemisphere ']

In [21]:
results_list = []

for title in title_list:
    
    url = 'https://marshemispheres.com/'
    browser.visit(url)  
    browser.links.find_by_partial_text(title).click()
    
    html = browser.html
    soup = bs(html, 'html.parser')
    results = soup.find_all('div', class_='downloads')
    
    results_list.append(results)
    
    for result in results:
        u_list = result.find('ul')
        l_list = u_list.find('li')
        link = l_list.find('a')
        href = link['href']
        imgurl_list.append(href)
    
    

In [16]:
results_list

[[<div class="downloads">
  <img class="thumb" src="images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/>
  <h3>Download</h3>
  <ul>
  <li><a href="images/full.jpg" target="_blank">Sample</a> (jpg) 1024px wide</li>
  <li><a href="images/cerberus_enhanced.tif" target="_blank">Original</a> (tif<span class="tooltip word-tif" title=""></span>) 21 MB</li>
  </ul>
  </div>],
 [<div class="downloads">
  <img class="thumb" src="images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png"/>
  <h3>Download</h3>
  <ul>
  <li><a href="images/schiaparelli_enhanced-full.jpg" target="_blank">Sample</a> (jpg) 1024px wide
                      </li>
  <li><a href="images/schiaparelli_enhanced.tif" target="_blank">Original</a> (tif<span class="tooltip word-tif" title=""></span>) 35 MB</li>
  </ul>
  </div>],
 [<div class="downloads">
  <img class="thumb" src="images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png"/>
  <h3>Download</h3>
  <ul>
  <

In [17]:
url = 'https://marshemispheres.com/cerberus.html'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

In [18]:
results = soup.find_all('div', class_='downloads')
print(results)

[<div class="downloads">
<img class="thumb" src="images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/>
<h3>Download</h3>
<ul>
<li><a href="images/full.jpg" target="_blank">Sample</a> (jpg) 1024px wide</li>
<li><a href="images/cerberus_enhanced.tif" target="_blank">Original</a> (tif<span class="tooltip word-tif" title=""></span>) 21 MB</li>
</ul>
</div>]


In [19]:
for result in results:
    u_list = result.find('ul')
    l_list = u_list.find('li')
    link = l_list.find('a')
    href = link['href']
    imgurl_list.append(href)

In [22]:
imgurl_list

['images/full.jpg',
 'images/full.jpg',
 'images/schiaparelli_enhanced-full.jpg',
 'images/syrtis_major_enhanced-full.jpg',
 'images/valles_marineris_enhanced-full.jpg']

In [ ]:
# Visit new site to find image jpg. for feature image
url = 'https://marshemispheres.com/'
browser.visit(url)

# Scrape site for featured image jpg.
# html = browser.html
# soup = bs(html, 'html.parser')

# results = soup.find_all('div', class_= 'description')
html = browser.html
soup = bs(html, 'html.parser')

results = soup.find_all('div', class_='description')

for result in results:
    title = result.find('h3').text
    print(title)
    
    browser.links.find_by_partial_text('Hemisphere').click()
    

    
#     title = result.find('h3')
#     print(title)
    
    
    
#     for result in results:
#         title = soup.find('article', class_='itemLink product-item').text
#         print(Title)
#     browser.links.find_by_partial_text('Hemisphere').click()
    
#     soup = bs(html, 'html.parser')
# #     results = soup.find_all('div', class_='downloads')
        
#         for result in results:
#             title = soup.find('div', class_='content_title').text
            
    


In [23]:
# # Quit browser once scraping is complete
browser.quit()